In [ ]:
##################################################################
#  Цель - отсканировать существующие препринты на сайте Банка России 
#     и собрать из этих них информацию о ключевых словах
# 1) Нужно научиться сканировать сайты и открывать пдф-файлы на этом сайте
# 2) Нужно научиться извлекать текст из этих pdf-файлов по заданному правилу
# 3) Нужно извлеченный текст соотнести с названием конкретного исследования в виде таблицы
# 4) Нужно экспортировать полученные данные в эксель формат
#
# Сайт 1: https://www.cbr.ru/ec_research/ser/
# Сайт 2: https://www.cbr.ru/dkp/system_p/
# Сайт 3: https://cbr.ru/ec_research/analitics/
#################################################################

In [1]:
# библиотеки
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [24]:
# целевые сайты
url1 = 'https://www.cbr.ru/ec_research/ser/'
# url2 = 'https://www.cbr.ru/dkp/system_p/research_notes/#a_119347'

r1 = requests.get(url1)
# r2 = requests.get(url2)

# чтобы начать парсить, нужно преобразовать информацию с сайта в объект "Soup"
soup1 = BeautifulSoup(r1.text, 'html')
# soup2 = BeautifulSoup(r2.text, 'html')

In [25]:
# находим все ссылки на исследования, которые расположены на сайте номер 1
base1 = soup1.find_all('a', class_ = 'referenceable')
# находим все ссылки на исследования, которые расположены на сайте номер 2
# base2 = soup2.find_all('a')


In [2]:
# URL 3
url3 = 'https://cbr.ru/ec_research/analitics/'

r3 = requests.get(url3)

# чтобы начать парсить, нужно преобразовать информацию с сайта в объект "Soup"
soup3 = BeautifulSoup(r3.text, 'html')

base3 = soup3.find_all('a')

In [3]:
# создаем словарь
research_data = []

In [ ]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 1
'''for link in base1:
    # извлекаем название и ссылку
    title_span = link.find('span', class_='document-regular_name_visible')
    if title_span:
        title = title_span.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })'''

In [ ]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 2
'''
for link in base2:
    # извлекаем значение атрибута href из ссылки
    href = link.get('href', '')
    if href and href.startswith('/StaticHtml') or href.startswith('/Content') :
        title = link.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })
        '''

In [4]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 3
for link in base3:
    # извлекаем значение атрибута href из ссылки
    href = link.get('href', '')
    if href and href.startswith('/Collection') or href.startswith('/Content') or href.startswith('/ec_research'):
        title = link.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [5]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Исследования
Ссылка: https://www.cbr.ru/ec_research/

2. 2024 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/55548/fs_review_2024.pdf

3. 2023 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/49075/fs_review_2023.pdf

4. 2023 год (графики)
Ссылка: https://www.cbr.ru/Collection/Collection/File/49074/charts_fs_review_2023.xlsx

5. 2024 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/55196/review_2024.pdf

6. 2024 год (графики)
Ссылка: https://www.cbr.ru/Collection/Collection/File/55197/review_2024.xlsx

7. 2023 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/48944/fi_review_2023.pdf

8. 2023 год (графики)
Ссылка: https://www.cbr.ru/Collection/Collection/File/48943/fi_review_2023.xlsx

9. 2022 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/43892/overview_2022.pdf

10. 2021 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/40903/overview_2021.pdf

11. 2021 год (графики)
Ссылка: https://www.cbr.ru/Collection/Collection/File/40902

### Теперь есть список статей и ссылки на них. Требуется из всех ссылок извлечь те, которые ведут на сайт, и заменить их на ссылки на pdf файлы

In [9]:
def replace_with_pdf(articles_list): # данная функция заменяет ссылки на статьи на прямые ссылки на pdf-файлы
    results = articles_list.copy() # финальный словарь, в который будут сохраняться финальные ссылки на pdf

    for i, article in enumerate(results):
        current_link = article['link'] # получаем текущую ссылку на статью
        print(f"Обрабатываем: {current_link}")

        if current_link.lower().endswith('.pdf'):
            print("Уже PDF, пропускаем")
            continue
        elif current_link.lower().endswith('.xlsx'):
            print("Excel файл, пропускаем")
            continue
        else:
            print("Уже исправляю ссылку на PDF")
            urltest = current_link
            rtest = requests.get(urltest)
            souptest = BeautifulSoup(rtest.text, 'html.parser')
            basetest = souptest.find_all('a')

            for i, link in enumerate(basetest, 1):
                href = link.get('href')
                if href and href.endswith('.pdf'):
                    if href.startswith('/'):
                        hrefpdf = f"https://www.cbr.ru{href}"
                        article['link'] = hrefpdf
                    elif href.startswith('http'):
                        hrefpdf = href
                        article['link'] = hrefpdf
                    else:
                        hrefpdf = f"https://www.cbr.ru/{href}"
                        article['link'] = hrefpdf

    seen_links = set()
    unique_research = []
    
    for research in results:
        link = research.get('link', '').strip()
        if link.endswith('.pdf'): # есть ссылки, которые ведут не на pdf, а на .xsls
            if link and link not in seen_links:
                seen_links.add(link)
                unique_research.append(research)
            elif not link:
                # Исследования без ссылки тоже добавляем
                unique_research.append(research)
        
    # Возвращаем список уникальных исследований
    return unique_research

research_data_pdf = replace_with_pdf(research_data)

Обрабатываем: https://www.cbr.ru/StaticHtml/File/119990/research_russia_04072023.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/55548/fs_review_2024.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/49075/fs_review_2023.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/49074/charts_fs_review_2023.xlsx
Excel файл, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/55196/review_2024.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/55197/review_2024.xlsx
Excel файл, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/48944/fi_review_2023.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/48943/fi_review_2023.xlsx
Excel файл, пропускаем
Обрабатываем: https://www.cbr.ru/Collection/Collection/File/43892/overview_2022.pdf
Уже PDF, пропускаем
Обрабатываем: https://www.cbr.ru/Collection

In [10]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data_pdf, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Исследования
Ссылка: https://www.cbr.ru/StaticHtml/File/119990/research_russia_04072023.pdf

2. 2024 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/55548/fs_review_2024.pdf

3. 2023 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/49075/fs_review_2023.pdf

4. 2024 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/55196/review_2024.pdf

5. 2023 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/48944/fi_review_2023.pdf

6. 2022 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/43892/overview_2022.pdf

7. 2021 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/40903/overview_2021.pdf

8. 2020 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/32168/overview_2020.pdf

9. 2019 год
Ссылка: https://www.cbr.ru/Collection/Collection/File/32167/overview_2019.pdf

10. Анализ предиктивных способностей опросных показателей Банка России, характеризующих рынок труда
Ссылка: https://www.cbr.ru/Content/Document/File/180583/analytic_note_20250

## Извлекаем ключевые слова из PDF

In [14]:
import PyPDF2
from io import BytesIO
import re
import pdfplumber

### *Напишем функцию, которая извлекает текст из pdf-файла*

In [15]:
def extract_text_from_pdf(pdf_link: str) -> str:
    try:
        response = requests.get(pdf_link)
        response.raise_for_status()
                
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            text = []
            total_pages = len(pdf.pages)
            for i in range(min(10, total_pages)):
                page = pdf.pages[i]  # Получаем объект страницы по индексу
                text.append(page.extract_text())

            # Извлекаем последние 4 страницы (если есть)
            if total_pages > 10:  # Чтобы не дублировать страницы
                start_last_pages = max(10, total_pages - 4)  # Начинаем с max(8, total_pages-4)
                for i in range(start_last_pages, total_pages):
                    page = pdf.pages[i]  # Получаем объект страницы по индексу
                    text.append(page.extract_text())
        return text

    except:
        response = requests.get(pdf_link)
        response.raise_for_status()

        pdf_data = BytesIO(response.content)

    
        reader = PyPDF2.PdfReader(pdf_data, strict=False)
        text = []

        total_pages = len(reader.pages)
        
        # Извлекаем первые 10 страниц
        for page_num in range(min(10, total_pages)):
            content = reader.pages[page_num].extract_text()
            text.append(content)
        
        # Извлекаем последние 4 страницы (если есть)
        if total_pages > 10:  # Чтобы не дублировать страницы
            start_last_pages = max(10, total_pages - 4)
            for page_num in range(start_last_pages, total_pages):
                content = reader.pages[page_num].extract_text()
                text.append(content)
            
        return text


### *Напишем функцию, которая очищает текст*

In [16]:
def clean_keywords_text(text: str) -> str: # данная функция очищает и восстанавливает текст
    proceeded_parts = []
    for part in text: # добавим точку в конец страницы, если ее нет
        if not part.endswith('.'):
                part += '.'
        proceeded_parts.append(part) 

    combined_text = " ".join(proceeded_parts) # объединяем все страницы вместе с пробелом
    lines = combined_text.split('\n') # разделяем текст на строки

    cleaned_lines = []
    for line in lines:
            # Убираем пробелы в начале и конце строки
            line = line.strip()
            cleaned_lines.append(line)

    full_text = ' '.join(cleaned_lines) # добавим пробел между строками

    full_text = re.sub(r'(\w+)-\s+', r'\1', full_text)
    full_text = re.sub(r'\s+', ' ', full_text)
    full_text = re.sub(r'\s+:', ':', full_text) # убираем пробел перед двоеточием
    full_text = re.sub(r'\s+ь', 'ь', full_text) # убираем пробел мягким знаком (в некоторых файлах при извлечении текста есть данная проблема)
    full_text = re.sub(r'\s+ы', 'ы', full_text)
    full_text = re.sub(r'\s+ъ', 'ъ', full_text)
    full_text = re.sub(r'\s+-', '-', full_text)
    full_text = re.sub(r'\s+»', '»', full_text)
    full_text = re.sub(r'\s+;', ';', full_text)
    full_text = re.sub(r':(\w)', r': \1', full_text) # добавляем пробел после двоеточия, если его нет

    return full_text.strip()

### *Напишем функцию, которая ищет в тексте ключевые слова*

In [17]:
def extract_keywords(text: str) -> str: # извлечение ключевых слов с учетом различных форматов:
    # варианты паттернов для поиска начала ключевых слов
    patterns = [
        r'Ключевые слова:*',
        r'Keywords:*',
        r'Key words:*',
        r'KEYWORDS:*',
        r'KEY WORDS:*',
        r'Ключевые слова\s*',  # Без двоеточия
        r'Keywords\s*',        # Без двоеточия
        r'Key words\s*',       # Без двоеточия
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            start_pos = match.end()
            # Находим позицию начала ключевых слов
            keywords_text = text[start_pos:]
            
            # Разбиваем текст на токены для анализа
            tokens = []
            current_token = ""
            
            # Проходим по каждому символу для анализа структуры
            for char in keywords_text:
                if char in [':', '.']:  # Сначала проверяем точку и двоеточие (если они есть, то значит ключевые слова 100% уже перечислены)
                    if current_token:
                        tokens.append(current_token)  # Добавляем последнее слово
                    break  # Выходим из цикла

                elif char in [',', ';'] or char.isspace():
                    if current_token:
                        tokens.append(current_token)
                        current_token = ""
                    tokens.append(char)
                else:
                    current_token += char


            # Собираем ключевые слова до условия остановки
            result_tokens = []
            prev_comma = False  # Флаг, что предыдущий токен был запятой
            
            for i, token in enumerate(tokens):
                if token and token[0].isupper() and prev_comma:
                    # Слово с заглавной буквы после запятой - вероятно начало нового раздела
                    # Проверяем, не является ли это частью ключевого слова (например, "ВВП")
                    if len(token) > 1 and not any(c.islower() for c in token[1:]):
                        # Если все остальные буквы заглавные (акроним), то продолжаем
                        result_tokens.append(token)
                    else:
                        # Останавливаемся на слове с заглавной буквы после запятой
                        break

                elif token.lower().startswith('jel') or token.lower().startswith('j.e.l'):
                    # начало JEL классификации
                    break
                    
                else:
                    result_tokens.append(token)
                
                # проверяем, был ли предыдущий токен запятой
                prev_comma = (token == ',')
            
            # Собираем результат
            keywords_result = "".join(result_tokens).strip()
            
            # Очищаем результат от лишних символов в конце
            keywords_result = re.sub(r'[,\s]+$', '', keywords_result)  # Убираем запятые и пробелы в конце
            keywords_result = re.sub(r'\s+,\s*', ', ', keywords_result)  # Нормализуем запятые
            
            return keywords_result
        
    return 'Нет ключевых слов'

In [ ]:
# проблемные места (первая ссылка):
#1) 41 статья: Коды?
#2) 32 статья: Классификация
#3) 59 статья: Коды

## Применим функции к полученным ранее ссылкам

In [18]:
def final_keywords_formation(data):
    for i, research in enumerate(data, 1):
        extracted_text = extract_text_from_pdf(research['link'])
        clean_text = clean_keywords_text(extracted_text)
        keywords = extract_keywords(clean_text) 
        research['keywords_total'] = keywords #список всех ключевых слов

        keyword_list = [kw.strip() for kw in re.split(r'[,;]', keywords) if kw.strip()] # разделяю список на отдельные ключевые слова
        # добавляю каждое ключевое слово в отдельный столбец
        for j, keyword in enumerate(keyword_list, 1):
            research[f'keyword_{j}'] = keyword
        
        print(f"Препринт {i}: {keywords}")
    return data

In [19]:
finally_keywords = final_keywords_formation(research_data_pdf)

Препринт 1: Нет ключевых слов
Препринт 2: Нет ключевых слов
Препринт 3: Нет ключевых слов
Препринт 4: Нет ключевых слов
Препринт 5: Нет ключевых слов
Препринт 6: Нет ключевых слов
Препринт 7: Нет ключевых слов
Препринт 8: Нет ключевых слов
Препринт 9: Нет ключевых слов


Cannot set gray non-stroke color because /'P19' is an invalid float value


Препринт 10: опросы предприятий, опережающие индикаторы, рынок труда, занятость


Cannot set gray non-stroke color because /'P26' is an invalid float value


Препринт 11: инвестиции в основной капитал, опросы предприятий, опережающие индикаторы, метод главных компонент
Препринт 12: секьюритизация, Asset-Backed Securities, мировой финансовый кризис


...

Препринт 89: Нет ключевых слов
Препринт 90: Нет ключевых слов
Препринт 91: Нет ключевых слов
Препринт 92: Нет ключевых слов
Препринт 93: (униграммы), на основе которых мы самостоятельно должны «назвать» каждую из перечисленных тем
Препринт 94: Нет ключевых слов
Препринт 95: Нет ключевых слов
Препринт 96: Нет ключевых слов


Cannot set gray non-stroke color because /'P164' is an invalid float value
Cannot set gray non-stroke color because /'P166' is an invalid float value
Cannot set gray non-stroke color because /'P168' is an invalid float value
Cannot set gray non-stroke color because /'P171' is an invalid float value
Cannot set gray non-stroke color because /'P173' is an invalid float value
Cannot set gray non-stroke color because /'P179' is an invalid float value
Cannot set gray non-stroke color because /'P181' is an invalid float value
Cannot set gray non-stroke color because /'P183' is an invalid float value
Cannot set gray non-stroke color because /'P186' is an invalid float value
Cannot set gray non-stroke color because /'P188' is an invalid float value
Cannot set gray non-stroke color because /'P189' is an invalid float value
Cannot set gray non-stroke color because /'P193' is an invalid float value
Cannot set gray non-stroke color because /'P195' is an invalid float value
Cannot set gray non-strok

Препринт 97: Нет ключевых слов
Препринт 98: Нет ключевых слов


Cannot set gray non-stroke color because /'P63' is an invalid float value
Cannot set gray non-stroke color because /'P65' is an invalid float value
Cannot set gray non-stroke color because /'P68' is an invalid float value
Cannot set gray non-stroke color because /'P89' is an invalid float value
Cannot set gray non-stroke color because /'P91' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P94' is an invalid float value
Cannot set gray non-stroke color because /'P119' is an invalid float value
Cannot set gray non-stroke color because /'P121' is an invalid float value


Препринт 99: Нет ключевых слов
Препринт 100: Нет ключевых слов
Препринт 101: Нет ключевых слов
Препринт 102: Нет ключевых слов
Препринт 103: Нет ключевых слов
Препринт 104: Нет ключевых слов
Препринт 105: Нет ключевых слов
Препринт 106: Нет ключевых слов
Препринт 107: Нет ключевых слов
Препринт 108: Нет ключевых слов
Препринт 109: Нет ключевых слов
Препринт 110: Нет ключевых слов
Препринт 111: Нет ключевых слов
Препринт 112: Нет ключевых слов
Препринт 113: Нет ключевых слов
Препринт 114: Нет ключевых слов
Препринт 115: Нет ключевых слов
Препринт 116: Нет ключевых слов
Препринт 117: Нет ключевых слов
Препринт 118: Нет ключевых слов
Препринт 119: Нет ключевых слов
Препринт 120: Нет ключевых слов
Препринт 121: Нет ключевых слов


# Экспортируем результаты в CSV

In [20]:
import csv

In [21]:
df = pd.DataFrame(finally_keywords)
df.to_csv('Preprints_Links_keywords', index = False, encoding='utf-8')
print("Данные экспортированы в Preprints_Links_keywords")

Данные экспортированы в Preprints_Links_keywords


#### Функции ниже предназначены для обнаружения и исправления ошибок в коде:

In [ ]:
text = extract_text_from_pdf(research_data_pdf[22]['link'])
text

Cannot set gray non-stroke color because /'P13' is an invalid float value
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from 

In [ ]:
clean = clean_keywords_text(text)
clean

'Неопределенность технического прогресса, байесовское обучение и «зеленый» парадокс Серия докладов об экономических исследованиях №. 133 / август 2024 А.А. Реентович. Александр Реентович Банк России, Департамент исследований и прогнозирования E-mail: ReentovichAA@cbr.ru Доклады Банка России проходят процедуру анонимного реЯцеНнзВирАовРанЬия 2с0о1 9ст ороны членов Консультативного исследовательского совета Банка России и внешних рецензентов. Автор благодарен Андрею Синякову за привлечение внимания автора к теме исследования. Автор также благодарит Ксению Юдаеву, Константина Стырина, Марка Сидоровского, анонимного рецензента и участников двух исследовательских семинаров Банка России за ценные комментарии и предложения. Автор благодарит Александра Цемахмана за подготовку русского текста. Содержание настоящего доклада по экономическим исследованиям отражает личную позицию авторов. Результаты исследования являются предварительными и публикуются с целью стимулировать обсуждение и получить ко